In [1]:
import torch
import numpy as np
from datasets import load_dataset
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader

/home/arjun/Desktop/AI_ENV/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# device = torch.device('cpu')
batch_size = 256

# Creating vocabulory

In [3]:
n = 3
embedding_dims = 10 # how many dimensional vector should represent each word in the vocabulory

In [4]:
ds = load_dataset("MuskumPillerum/General-Knowledge")
df = ds['train'][:10]

In [5]:
def tokenize(sentence):
    data = sentence
    split_terms = [',', '.', '!', '?', '(', ')', '&', '$', '+', '-', '/', '*', ';', ':']
    for split_term in split_terms:
        if split_term in sentence:
            data = data.replace(split_term, f' {split_term} ')
    data = data.split()
    return data

tokenize('what is your !! ! name ?    ')

['what', 'is', 'your', '!', '!', '!', 'name', '?']

In [6]:
vocab_list = set()
X = []
for x,y in zip(df['Question'], df['Answer']):
    data = f'Question: {x} Answer: {y}'
    data = data.lower().replace('\\n', '')
    vocab_list.update(tokenize(data))
    X.append(data)

In [7]:
vocab_list.add('<UNK>')

In [8]:
vocab = {v:i for v,i in zip(vocab_list, range(0, len(vocab_list)+1))}
len(vocab)

141

In [9]:
vocab

{'brain': 0,
 'world': 1,
 'unsupervised': 2,
 'learns': 3,
 'computer': 4,
 'answer': 5,
 'ai': 6,
 'with': 7,
 'computing': 8,
 'inspired': 9,
 'analyze': 10,
 'is': 11,
 'form': 12,
 '?': 13,
 'are': 14,
 'from': 15,
 'given': 16,
 'takes': 17,
 '.': 18,
 'narrow': 19,
 'ability': 20,
 'perform': 21,
 'decisions': 22,
 'based': 23,
 'maximize': 24,
 'information': 25,
 'neurons': 26,
 'without': 27,
 'structure': 28,
 'computers': 29,
 'systems': 30,
 'that': 31,
 'a': 32,
 'data': 33,
 'interconnected': 34,
 'designed': 35,
 'using': 36,
 'intelligence': 37,
 'processing': 38,
 'such': 39,
 'this': 40,
 'network': 41,
 'consists': 42,
 'by': 43,
 'where': 44,
 'speech': 45,
 'relationships': 46,
 'system': 47,
 'it': 48,
 'in': 49,
 'of': 50,
 'as': 51,
 'generate': 52,
 'algorithms': 53,
 'natural': 54,
 'uses': 55,
 'time': 56,
 'categories': 57,
 'development': 58,
 'machine': 59,
 'main': 60,
 'language': 61,
 'function': 62,
 'refers': 63,
 'focuses': 64,
 'improve': 65,
 '<UN

In [10]:
embedding_matrix = torch.randn(len(vocab),embedding_dims)
embedding_matrix.shape

torch.Size([141, 10])

In [11]:
embedding_matrix[[1,2,3]]

tensor([[ 0.1441, -0.0660, -0.1486, -0.5367,  0.8855,  0.3319,  2.3894, -0.0252,
          1.1656,  0.4430],
        [ 0.5385,  0.4685,  1.5679,  1.0924,  1.7072, -0.7975,  0.8017, -0.6557,
         -0.0391,  1.8227],
        [ 1.6989,  0.8181,  0.5669, -0.5853,  0.3487, -0.5965, -0.2985,  0.5015,
         -0.1862,  1.2021]])

In [12]:
def get_word_embedding(word, vocab=vocab, embedding_matrix=embedding_matrix):
    if word not in vocab:
        word = '<UNK>'
    embedding = embedding_matrix[vocab[word]]
    return embedding

get_word_embedding('as')

tensor([ 0.6150,  2.1484, -0.3243, -0.1649, -0.1562, -0.3681, -0.5178,  0.6161,
        -0.9528, -0.5300])

# Creating the model

In [13]:
from print_color import print

In [14]:
class NeuralNetwork(nn.Module):
    def __init__(self, n=n, num_hidden_layer=1024, vocab_len = len(vocab_list), dim_embedding=10):
        super(NeuralNetwork, self).__init__()
        self.n = n
        self.dim_embedding = dim_embedding
        self.embedding = nn.Embedding(vocab_len, dim_embedding).to(device)

        self.hidden_layer = nn.Linear((n-1)*dim_embedding, num_hidden_layer)
        self.relu = nn.ReLU()
        self.output = nn.Linear(num_hidden_layer, vocab_len)
        # self.softmax = nn.Softmax(dim=1)

    def forward(self,x):
        # x will be the indices of embedding representing the input words
        x_embeddings = self.embedding(x).view(-1,(self.n-1)*self.dim_embedding).to(device)
        out = self.hidden_layer(x_embeddings)
        out = self.relu(out)
        out = self.output(out)
        return out

model = NeuralNetwork(n=3).to(device)

# Creating dataset

In [15]:
dataset = []
for x,y in zip(df['Question'], df['Answer']):
    data = f'Question: {x} Answer: {y}'.lower()
    tokenized_data = tokenize(data)
    for i in range(len(tokenized_data)-n):
        # print(i)
        data_i = tokenized_data[i:i+n]
        dataset.append([vocab[i] if i in vocab else vocab['<UNK>'] for i in data_i])
    # print()

dataset_np = np.array(dataset)
dataset_np.shape

(419, 3)

In [16]:
class QuestionAnswerDataset(Dataset):
    def __init__(self, dataset):
        self.dataset = dataset
        self.x = dataset[:,[i for i in range(n-1)]]
        self.y = dataset[:,-1]
        self.m, self.n = self.x.shape
    
    def __getitem__(self, index):
        return self.x[index], self.y[index]
    
    def __len__(self):
        return self.m
    
dataset = QuestionAnswerDataset(dataset=dataset_np)
dataloader = DataLoader(dataset=dataset, batch_size=batch_size, shuffle=True, num_workers=0)

# Training

In [17]:
num_epochs = 100
lr = 0.01
print(device)

cuda


In [18]:
model = model.to(device)
model = torch.compile(model)

In [19]:
lossCategory = nn.CrossEntropyLoss()
optimiser = torch.optim.SGD(model.parameters(), lr=lr)

In [29]:
for epoch in range(num_epochs):
    for i, (input, label) in enumerate(dataloader):
        x = input.to(device)
        y = label.to(device)
        y_pred = model(x)
        loss = lossCategory(y_pred, y)
        if epoch%10==0 and i==0: 
            # print(epoch)
            print(loss.item())
        loss.backward()
        optimiser.step()
        optimiser.zero_grad()


1.575454592704773
1.535890817642212
1.4748655557632446
1.5005031824111938
1.442416787147522
1.4320796728134155
1.4392311573028564
1.429460048675537
1.379377007484436
1.4494707584381104


# Inference

In [37]:
text = 'Who is the king?'
tokenized_text = tokenize(text)
embedding_indices = [get_word_embedding(word) for word in tokenized_text]
embedding_indices
# convert to tensor object


[tensor([-1.2157,  1.4776, -1.6006,  0.8786, -0.7282,  0.3264,  0.3275, -1.3843,
          2.0836, -0.1157]),
 tensor([ 1.4275,  0.4805, -0.2581, -0.7341, -0.3106,  0.2474, -0.3129, -0.4852,
         -0.3051,  0.5465]),
 tensor([ 0.2556, -0.5062,  0.4376, -1.7617,  0.9487,  1.2253, -0.5775,  0.6699,
          0.7205, -0.3678]),
 tensor([-1.2157,  1.4776, -1.6006,  0.8786, -0.7282,  0.3264,  0.3275, -1.3843,
          2.0836, -0.1157]),
 tensor([-0.0436, -1.1707,  0.1053, -0.6188, -0.6143,  0.5261,  2.4051, -1.2662,
         -0.7975,  0.9164])]

In [21]:
# TO DO
# normalisation, standardise, proper init, weight decay, hyperparameter tunign, diff optimiser test, inference, earlyy stopping
# add direct connections, see if it is useful
# plot unigram, bi, tri, etc perplexity score, word error rate, etc